In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score
import graphviz, pydotplus
from sklearn.tree import export_graphviz
from prettytable import PrettyTable

In [2]:
def extractData(csvFile):
    df = pd.read_csv(csvFile)
    y = df['label']
    X = df.drop(columns = ['accountName', 'label', 'Unnamed: 0'])
    print(X.columns)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42, stratify = y)
    scaler = StandardScaler()
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    
    return scaler, X_train, X_test, y_train, y_test

csv_file_name = 'all_accounts_filtered_w2v.csv'
main_dir_path = os.path.dirname(os.getcwd())
csv_file_path = os.path.join(main_dir_path, 'Data', 'FINAL_DATASETS', csv_file_name)
scaler, X_train, X_test, y_train, y_test = extractData(csv_file_path)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

Index(['daily_downtime', 'uniqueDomains', 'uniqueSubreddits', 'uniqueTitles',
       'numSubmissions', 'top_1', 'top_2', 'top_3', 'uniqueComments',
       'SubmToCommentRatio', 'title_w2v_1', 'title_w2v_2', 'title_w2v_3',
       'title_w2v_4', 'title_w2v_5', 'title_w2v_6', 'title_w2v_7',
       'title_w2v_8', 'title_w2v_9', 'title_w2v_10', 'title_w2v_11',
       'title_w2v_12', 'title_w2v_13', 'title_w2v_14', 'title_w2v_15',
       'title_w2v_16', 'title_w2v_17', 'title_w2v_18', 'title_w2v_19',
       'title_w2v_20', 'comment_w2v_1', 'comment_w2v_2', 'comment_w2v_3',
       'comment_w2v_4', 'comment_w2v_5', 'comment_w2v_6', 'comment_w2v_7',
       'comment_w2v_8', 'comment_w2v_9', 'comment_w2v_10', 'comment_w2v_11',
       'comment_w2v_12', 'comment_w2v_13', 'comment_w2v_14', 'comment_w2v_15',
       'comment_w2v_16', 'comment_w2v_17', 'comment_w2v_18', 'comment_w2v_19',
       'comment_w2v_20'],
      dtype='object')


((469, 50), (83, 50), (469,), (83,))

In [3]:
clf_knn = KNeighborsClassifier()
k_list = [1, 5, 9, 13, 17, 21, 25, 31, 41, 51]
parameters = {'n_neighbors': k_list}
gs_cv_knn = GridSearchCV(clf_knn, parameters, cv = 3, scoring = 'accuracy')
gs_cv_knn.fit(X_train, y_train)
clf_knn = gs_cv_knn.best_estimator_

clf_knn.fit(X_train, y_train)
y_pred_knn = clf_knn.predict(X_test)
print(accuracy_score(y_test, y_pred_knn), f1_score(y_test, y_pred_knn), clf_knn)

0.9036144578313253 0.9069767441860465 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')


In [4]:
clf_lr = LogisticRegression(penalty= 'l1')
c_list = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
parameters = {'C' : c_list}
gs_cv_lr = GridSearchCV(clf_lr, parameters, cv = 3, scoring='accuracy')
gs_cv_lr.fit(X_train, y_train)
clf_lr = gs_cv_lr.best_estimator_

clf_lr.fit(X_train, y_train)
y_pred_lr = clf_lr.predict(X_test)
print(accuracy_score(y_test, y_pred_lr), f1_score(y_test, y_pred_lr), clf_lr)

/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver 

0.8554216867469879 0.8500000000000001 LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [5]:
clf_svm = SGDClassifier(loss= 'hinge', penalty='l2')
alpha_list = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
parameters = {'alpha' : alpha_list}
gs_cv_svm = GridSearchCV(clf_svm, parameters, cv = 3, scoring='accuracy')
gs_cv_svm.fit(X_train, y_train)
clf_svm = gs_cv_svm.best_estimator_

clf_svm.fit(X_train, y_train)
y_pred_svm = clf_svm.predict(X_test)
print(accuracy_score(y_test, y_pred_svm), f1_score(y_test, y_pred_svm), clf_svm)

0.891566265060241 0.8941176470588236 SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
              epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
              learning_rate='optimal', loss='hinge', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)


/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [6]:
clf_dt = DecisionTreeClassifier()
max_depth_list = [1, 5, 10, 50, 100, 500, 1000]
min_samples_split_list = [5, 10, 100, 500]
parameters = {'max_depth': max_depth_list, 'min_samples_split': min_samples_split_list}
gs_cv_dt = GridSearchCV(clf_dt, parameters, cv = 3, scoring='accuracy')
gs_cv_dt.fit(X_train, y_train)
clf_dt = gs_cv_dt.best_estimator_

clf_dt.fit(X_train, y_train)
y_pred_dt = clf_dt.predict(X_test)
print(accuracy_score(y_test, y_pred_dt), f1_score(y_test, y_pred_dt), clf_dt)

df = pd.read_csv(csv_file_path)
df = df.drop(columns = ['accountName', 'label', 'Unnamed: 0'])
dot_data = export_graphviz(clf_dt, out_file= None, feature_names= list(df.columns), filled= True)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png('DT_PLOTS/DecisionTreeVisualization_filtered_w2v.png')

0.9036144578313253 0.9024390243902439 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


True